# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'08-19-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'08-19-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head(20)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-08-20 04:27:43,33.939110,67.709953,37599,1375,27166,9058.0,Afghanistan,96.585159,3.657012
1,NaN,NaN,NaN,Albania,2020-08-20 04:27:43,41.153300,20.168300,7812,234,3928,3650.0,Albania,271.457363,2.995392
2,NaN,NaN,NaN,Algeria,2020-08-20 04:27:43,28.033900,1.659600,39847,1402,27971,10474.0,Algeria,90.868990,3.518458
3,NaN,NaN,NaN,Andorra,2020-08-20 04:27:43,42.506300,1.521800,1024,53,875,96.0,Andorra,1325.309001,5.175781
4,NaN,NaN,NaN,Angola,2020-08-20 04:27:43,-11.202700,17.873900,2015,92,698,1225.0,Angola,6.130906,4.565757
5,NaN,NaN,NaN,Antigua and Barbuda,2020-08-20 04:27:43,17.060800,-61.796400,94,3,88,3.0,Antigua and Barbuda,95.988890,3.191489
6,NaN,NaN,NaN,Argentina,2020-08-20 04:27:43,-38.416100,-63.616700,312659,6330,228725,77604.0,Argentina,691.788085,2.024570
7,NaN,NaN,NaN,Armenia,2020-08-20 04:27:43,40.069100,45.038200,42056,833,35226,5997.0,Armenia,1419.260173,1.980692
8,NaN,NaN,Australian Capital Territory,Australia,2020-08-20 04:27:43,-35.473500,149.012400,113,3,110,0.0,"Australian Capital Territory, Australia",26.395702,2.654867
9,NaN,NaN,New South Wales,Australia,2020-08-20 04:27:43,-33.868800,151.209300,3971,52,2995,924.0,"New South Wales, Australia",48.915989,1.309494


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,37162,37269,37345,37424,37431,37551,37596,37599,37599,37599
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,6536,6676,6817,6971,7117,7260,7380,7499,7654,7812
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,35712,36204,36699,37187,37664,38133,38583,39025,39444,39847


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1328,1344,1354,1363,1363,1370,1375,1375,1375,1375
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,200,205,208,213,219,225,228,230,232,234
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1312,1322,1333,1341,1351,1360,1370,1379,1391,1402


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,26228,26415,26694,26714,26714,27166,27166,27166,27166,27166
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3379,3480,3552,3616,3695,3746,3794,3816,3871,3928
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,24920,25263,25627,26004,26308,26644,27017,27347,27653,27971


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1174,1162,1188,1196,1196,1206,1218,1219,1235,1241
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,3651,3670,3710,3744,3791,3839,3877,3888,3906,3931
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,576,579,581,586,590,598,604,605,606,613


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,21,22,22,22,22,22,22,22,22,22
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,25,29,29,29,29,29,29,29,30,30
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,5,6,6,6,6,6,6,6,6,7


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
631,1001.0,Autauga,Alabama,US,2020-08-20 04:27:43,32.539527,-86.644082,1241,22,0,1219.0,"Autauga, Alabama, US",2221.267608,1.772764
668,1075.0,Lamar,Alabama,US,2020-08-20 04:27:43,33.779950,-88.096680,249,2,0,247.0,"Lamar, Alabama, US",1803.694314,0.803213
669,1077.0,Lauderdale,Alabama,US,2020-08-20 04:27:43,34.901719,-87.656247,1278,21,0,1257.0,"Lauderdale, Alabama, US",1378.209622,1.643192


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,5529824,173177,1925049
Brazil,3456652,111100,2801931
India,2836925,53866,2096664


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,5529824,173177,1925049,3431598,2020-08-20 04:27:43,37.919464,-91.363237
Brazil,3456652,111100,2801931,543621,2020-08-20 04:27:43,-12.669522,-48.480493
India,2836925,53866,2096664,686395,2020-08-20 04:27:43,23.088275,81.806127
Russia,935066,15951,747802,171313,2020-08-20 04:27:43,54.546312,62.120860
South Africa,596060,12423,491441,92196,2020-08-20 04:27:43,-30.559500,22.937500


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,647274,11690,0
Florida,584047,9932,0
Texas,574633,10934,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,647274,11690,0,635584,2020-08-20 04:27:43,37.843962,-120.728594
Florida,584047,9932,0,574115,2020-08-20 04:27:43,28.940755,-82.700744
Texas,574633,10934,0,563699,2020-08-20 04:27:43,31.660643,-98.653069
New York,427202,32865,0,394337,2020-08-20 04:27:43,42.671593,-75.579694
Georgia,243982,4849,0,239133,2020-08-20 04:27:43,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,231015,23643,0
California,Los Angeles,225827,5392,0
Florida,Miami-Dade,148093,2166,0
Arizona,Maricopa,130384,2697,0
Illinois,Cook,118012,4984,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,231015,23643,0,207372,2020-08-20 04:27:43,40.767273,-73.971526,36061.0
California,Los Angeles,225827,5392,0,220435,2020-08-20 04:27:43,34.308284,-118.228241,6037.0
Florida,Miami-Dade,148093,2166,0,145927,2020-08-20 04:27:43,25.611236,-80.551706,12086.0
Arizona,Maricopa,130384,2697,0,127687,2020-08-20 04:27:43,33.348359,-112.491815,4013.0
Illinois,Cook,118012,4984,0,113028,2020-08-20 04:27:43,41.841448,-87.816588,17031.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-17,37599,7499,39025,1005,1935,93,299126,41701,23773,23534,...,321064,1457,35702,34802,983,16844,10,1882,9839,5308
2020-08-18,37599,7654,39444,1005,1966,93,305966,41846,23989,23829,...,322177,1485,36352,35697,989,17306,10,1889,9981,5378
2020-08-19,37599,7812,39847,1024,2015,94,312659,42056,24236,24084,...,323008,1493,37112,36868,994,17606,10,1892,10218,5643


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-17,1375,230,1379,53,88,3,5814,824,438,729,...,41454,40,236,288,25,113,1,535,264,135
2020-08-18,1375,232,1391,53,90,3,6048,832,450,729,...,41466,40,242,297,26,113,1,537,264,141
2020-08-19,1375,234,1402,53,92,3,6330,833,463,729,...,41483,40,248,303,25,119,1,539,269,150


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-17,27166,3816,27347,869,632,88,217850,34655,14536,20765,...,1490,1205,30973,23575,467,9906,8,1045,8575,3848
2020-08-18,27166,3871,27653,869,667,88,223531,34982,14927,20870,...,1493,1219,32062,24561,526,9939,8,1052,8776,4105
2020-08-19,27166,3928,27971,875,698,88,228725,35226,15247,20958,...,1495,1228,32944,25416,533,10312,8,1055,9126,4442


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1174,1162,1188,1196,1196,1206,1218,1219,1235,1241
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,229,230,230,237,237,239,240,245,247,249
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1189,1190,1203,1213,1219,1231,1238,1240,1269,1278
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,351,354,356,360,364,367,370,370,377,379
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,2700,2708,2724,2736,2735,2760,2776,2779,2861,2888


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-17,109004,4309,0,194005,53077,629415,53351,51267,16536,49,...,134744,561110,46894,1527,760,107421,67721,8641,66196,3331
2020-08-18,110361,4371,0,194920,53487,638140,53612,51255,16593,49,...,135770,568812,47157,1530,796,108282,68263,8738,66196,3360
2020-08-19,111478,4438,0,195557,54216,647274,53882,51314,16643,49,...,137800,574633,47521,1533,828,109019,68689,8807,67493,3431


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-17        1219    3888     605  474    863     496    795    1936   
2020-08-18        1235    3906     606  476    887     504    796    1982   
2020-08-19        1241    3931     613  490    901     522    796    2028   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-17          855      306  ...     146      6       86       40   
2020-08-18          859      313  ...     148      7       91       40   
2020-08-19          859      313  ...     152      7      101       41   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-17            278   390   277          0      100      8  
2020-08-18            279   393   278          0      102      8  
2020-08-19            285   394   283          0      104     12  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-17,1925,28,0,4506,603,11296,1896,4456,593,0,...,1387,10447,364,58,9,2385,1785,160,1039,30
2020-08-18,1936,29,0,4529,619,11504,1899,4456,593,0,...,1424,10678,369,58,9,2396,1808,162,1039,30
2020-08-19,1944,29,0,4634,631,11690,1900,4457,595,0,...,1452,10934,377,58,9,2410,1822,166,1060,34


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-17          22      29       6    5      5      13     36      21   
2020-08-18          22      30       6    5      5      13     36      23   
2020-08-19          22      30       7    5      5      13     36      23   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-17           38        8  ...       0      0        0        0   
2020-08-18           38        8  ...       0      0        0        0   
2020-08-19           38        8  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-17              0     0     0         29        0      0  
2020-08-18              0     0     0         29        0      0  
2020-08-19              0     0     0         33        0      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-17,0.00008,0.016125,0.011456,0.016178,0.015215,0.000000,0.015470,0.000912,0.009126,0.007018,...,0.002251,0.011806,0.010558,0.031018,0.019710,0.018749,0.0,0.006956,0.053088,0.008934
2020-08-18,0.00000,0.020669,0.010737,0.000000,0.016021,0.000000,0.022867,0.003477,0.009086,0.012535,...,0.003467,0.019218,0.018206,0.025717,0.006104,0.027428,0.0,0.003719,0.014432,0.013188
2020-08-19,0.00000,0.020643,0.010217,0.018905,0.024924,0.010753,0.021875,0.005018,0.010296,0.010701,...,0.002579,0.005387,0.020907,0.032804,0.005056,0.017335,0.0,0.001588,0.023745,0.049275


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-17,0.0,0.008772,0.006569,0.0,0.000000,0.0,0.019463,0.007335,0.040380,0.001374,...,0.000072,0.052632,0.017241,0.024911,0.041667,0.027273,0.0,0.009434,0.015385,0.022727
2020-08-18,0.0,0.008696,0.008702,0.0,0.022727,0.0,0.040248,0.009709,0.027397,0.000000,...,0.000289,0.000000,0.025424,0.031250,0.040000,0.000000,0.0,0.003738,0.000000,0.044444
2020-08-19,0.0,0.008621,0.007908,0.0,0.022222,0.0,0.046627,0.001202,0.028889,0.000000,...,0.000410,0.000000,0.024793,0.020202,-0.038462,0.053097,0.0,0.003724,0.018939,0.063830


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-17,0.0,0.005799,0.012215,0.006952,0.006369,0.0,0.029041,0.002053,0.032166,0.004062,...,0.002692,0.004167,0.000000,0.038546,0.024123,0.006912,0.0,0.031589,0.019377,0.839388
2020-08-18,0.0,0.014413,0.011190,0.000000,0.055380,0.0,0.026078,0.009436,0.026899,0.005057,...,0.002013,0.011618,0.035160,0.041824,0.126338,0.003331,0.0,0.006699,0.023440,0.066788
2020-08-19,0.0,0.014725,0.011500,0.006904,0.046477,0.0,0.023236,0.006975,0.021438,0.004217,...,0.001340,0.007383,0.027509,0.034811,0.013308,0.037529,0.0,0.002852,0.039881,0.082095


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-17,0.005266,0.011740,NaN,0.002418,0.007823,0.008883,0.003650,0.007270,0.005167,0.0,...,0.007748,0.010292,0.005187,0.007921,0.025641,0.006880,0.003854,0.008638,0.006921,0.013694
2020-08-18,0.012449,0.014388,NaN,0.004716,0.007725,0.013862,0.004892,-0.000234,0.003447,0.0,...,0.007614,0.013726,0.005608,0.001965,0.047368,0.008015,0.008003,0.011226,0.000000,0.008706
2020-08-19,0.010121,0.015328,NaN,0.003268,0.013629,0.014313,0.005036,0.001151,0.003013,0.0,...,0.014952,0.010234,0.007719,0.001961,0.040201,0.006806,0.006241,0.007897,0.019593,0.021131


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-17      0.000821  0.002837  0.001656  0.010661  0.003488  0.000000   
2020-08-18      0.013126  0.004630  0.001653  0.004219  0.027810  0.016129   
2020-08-19      0.004858  0.006400  0.011551  0.029412  0.015784  0.035714   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-08-17      0.003788  0.008859  0.001171  0.013245  ...  0.013889   
2020-08-18      0.001258  0.023760  0.004678  0.022876  ...  0.013699   
2020-08-19      0.000000  0.023209  0.000000  0.000000  ...  0.027027   

Province_State                                                              \
Admin2            Platte  Sheridan Sublette Sweetwater     Teton     Uinta   
2020-08-17      0.000000  0.036145    0.000   0.007246  0.002571  0.003623   
2020-08-18      0.166667  0.058140    0.000   0.003597  0.007692  0.003610   
2020-08-19      0.000000  0.109890    0.025   0.021505  0.002545  0.017986   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-08-17            NaN  0.030928  0.142857  
2020-08-18            NaN  0.020000  0.000000  
2020-08-19            NaN  0.019608  0.500000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-17,0.014226,0.000000,NaN,0.000000,0.006678,0.004714,0.000000,0.000674,0.000000,NaN,...,0.015373,0.004906,0.002755,0.0,0.0,0.001680,0.002246,0.000000,0.000000,0.000000
2020-08-18,0.005714,0.035714,NaN,0.005104,0.026534,0.018414,0.001582,0.000000,0.000000,NaN,...,0.026676,0.022112,0.013736,0.0,0.0,0.004612,0.012885,0.012500,0.000000,0.000000
2020-08-19,0.004132,0.000000,NaN,0.023184,0.019386,0.016168,0.000527,0.000224,0.003373,NaN,...,0.019663,0.023975,0.021680,0.0,0.0,0.005843,0.007743,0.024691,0.020212,0.133333


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                   \
Admin2         Autauga   Baldwin   Barbour Bibb Blount   Bullock Butler   
2020-08-17         0.0  0.000000  0.000000  0.0    0.0  0.181818    0.0   
2020-08-18         0.0  0.034483  0.000000  0.0    0.0  0.000000    0.0   
2020-08-19         0.0  0.000000  0.166667  0.0    0.0  0.000000    0.0   

Province_State                              ... Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2020-08-17      0.105263      0.0      0.0  ...     NaN    NaN      NaN   
2020-08-18      0.095238      0.0      0.0  ...     NaN    NaN      NaN   
2020-08-19      0.000000      0.0      0.0  ...     NaN    NaN      NaN   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-17          NaN        NaN   NaN   NaN   0.000000      NaN    NaN  
2020-08-18          NaN        NaN   NaN   NaN   0.000000      NaN    NaN  
2020-08-19          NaN        NaN   NaN   NaN   0.137931      NaN    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-17,0.000887,0.017378,0.011892,0.008994,0.015946,0.000680,0.018614,0.002789,0.010621,0.008438,...,0.002909,0.009266,0.016019,0.033915,0.019047,0.020487,7.709882e-19,0.005591,0.036965,0.013308
2020-08-18,0.000444,0.019024,0.011314,0.004497,0.015984,0.000340,0.020741,0.003133,0.009853,0.010487,...,0.003188,0.014242,0.017113,0.029816,0.012576,0.023957,3.854941e-19,0.004655,0.025699,0.013248
2020-08-19,0.000222,0.019833,0.010766,0.011701,0.020454,0.005546,0.021308,0.004076,0.010075,0.010594,...,0.002884,0.009814,0.019010,0.031310,0.008816,0.020646,1.927471e-19,0.003122,0.024722,0.031261


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-17,0.002029,0.014199,0.006859,0.000300,0.011082,2.369356e-38,0.019239,0.005119,0.044800,0.001282,...,0.000118,0.028007,0.022088,0.026033,0.044114,0.021572,0.0,0.005928,0.012971,0.021563
2020-08-18,0.001014,0.011447,0.007780,0.000150,0.016905,1.184678e-38,0.029744,0.007414,0.036099,0.000641,...,0.000204,0.014004,0.023756,0.028641,0.042057,0.010786,0.0,0.004833,0.006486,0.033004
2020-08-19,0.000507,0.010034,0.007844,0.000075,0.019563,5.923390e-39,0.038185,0.004308,0.032494,0.000320,...,0.000307,0.007002,0.024275,0.024422,0.001798,0.031942,0.0,0.004279,0.012712,0.048417


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-17,0.002404,0.010431,0.012863,0.004256,0.013594,0.020822,0.034881,0.004127,0.028708,0.004253,...,0.002128,0.005453,0.025953,0.035742,0.022558,0.020620,4.517509e-21,0.021046,0.031605,0.437831
2020-08-18,0.001202,0.012422,0.012026,0.002128,0.034487,0.010411,0.030479,0.006781,0.027803,0.004655,...,0.002071,0.008536,0.030556,0.038783,0.074448,0.011976,2.258755e-21,0.013872,0.027523,0.252309
2020-08-19,0.000601,0.013573,0.011763,0.004516,0.040482,0.005206,0.026858,0.006878,0.024621,0.004436,...,0.001705,0.007959,0.029033,0.036797,0.043878,0.024752,1.129377e-21,0.008362,0.033702,0.167202


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-17,0.007111,0.017518,NaN,0.003633,0.007739,0.010292,0.005017,0.003869,0.005842,1.490743e-47,...,0.010655,0.012645,0.006550,0.006603,0.026966,0.008079,0.006838,0.012151,0.009490,0.014939
2020-08-18,0.009780,0.015953,NaN,0.004175,0.007732,0.012077,0.004955,0.001817,0.004645,7.453715e-48,...,0.009135,0.013186,0.006079,0.004284,0.037167,0.008047,0.007421,0.011688,0.004745,0.011823
2020-08-19,0.009951,0.015641,NaN,0.003721,0.010681,0.013195,0.004995,0.001484,0.003829,3.726858e-48,...,0.012043,0.011710,0.006899,0.003122,0.038684,0.007427,0.006831,0.009792,0.012169,0.016477


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-17      0.004622  0.006934  0.005846  0.008267  0.006071  0.001607   
2020-08-18      0.008874  0.005782  0.003750  0.006243  0.016940  0.008868   
2020-08-19      0.006866  0.006091  0.007650  0.017828  0.016362  0.022291   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-08-17      0.005135  0.010101  0.001244  0.014081  ...  0.010208   
2020-08-18      0.003196  0.016931  0.002961  0.018478  ...  0.011953   
2020-08-19      0.001598  0.020070  0.001481  0.009239  ...  0.019490   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-08-17      0.006250  0.032932  0.000404   0.007623  0.006107  0.001159   
2020-08-18      0.086458  0.045536  0.000202   0.005610  0.006900  0.002385   
2020-08-19      0.043229  0.077713  0.012601   0.013558  0.004722  0.010185   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-08-17           -1.0  0.026666  0.120908  
2020-08-18           -1.0  0.023333  0.060454  
2020-08-19           -1.0  0.021470  0.280227  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-17,0.008423,0.005271,NaN,0.004235,0.007249,0.006096,0.000916,0.000403,0.000087,NaN,...,0.015118,0.011308,0.004141,0.000004,0.000031,0.001878,0.005133,0.005215,0.002633,0.001376
2020-08-18,0.007068,0.020493,NaN,0.004670,0.016892,0.012255,0.001249,0.000202,0.000043,NaN,...,0.020897,0.016710,0.008939,0.000002,0.000015,0.003245,0.009009,0.008857,0.001316,0.000688
2020-08-19,0.005600,0.010246,NaN,0.013927,0.018139,0.014211,0.000888,0.000213,0.001708,NaN,...,0.020280,0.020342,0.015309,0.000001,0.000008,0.004544,0.008376,0.016774,0.010764,0.067011


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-17      0.000378  0.001376  0.001563  0.000292  0.002281  0.090909   
2020-08-18      0.000189  0.017930  0.000781  0.000146  0.001140  0.045455   
2020-08-19      0.000095  0.008965  0.083724  0.000073  0.000570  0.022727   

Province_State                                              ... Wyoming  \
Admin2            Butler   Calhoun      Chambers  Cherokee  ...    Park   
2020-08-17      0.000028  0.054978  9.316438e-09  0.000558  ...     NaN   
2020-08-18      0.000014  0.075108  4.658219e-09  0.000279  ...     NaN   
2020-08-19      0.000007  0.037554  2.329109e-09  0.000140  ...     NaN   

Province_State                                                             \
Admin2         Platte Sheridan Sublette Sweetwater Teton Uinta Unassigned   
2020-08-17        NaN      NaN      NaN        NaN   NaN   NaN   0.001425   
2020-08-18        NaN      NaN      NaN        NaN   NaN   NaN   0.000713   
2020-08-19        NaN      NaN      NaN        NaN   NaN   NaN   0.069322   

Province_State                  
Admin2         Washakie Weston  
2020-08-17          NaN    NaN  
2020-08-18          NaN    NaN  
2020-08-19          NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200819,AK,5247,305602,NaN,56.0,NaN,NaN,NaN,6.0,...,310849,0,0,85dcc331b8b35c88c030526eaedc78fce146bef2,0,0,0,0,0,NaN
1,20200819,AL,111478,773868,NaN,1198.0,13080.0,NaN,1336.0,NaN,...,885346,8,122,7095d1d867307ff5170d94a36cffbe87e3dbc113,0,0,0,0,0,NaN
2,20200819,AR,54216,587064,NaN,499.0,3743.0,NaN,NaN,114.0,...,641280,12,48,6a1605093a5772590825ab1faa2581c75ba85fef,0,0,0,0,0,NaN
3,20200819,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,c0f8fafd05d99956acbdcaa7c6515503d8e27f8f,0,0,0,0,0,NaN
4,20200819,AZ,195557,915682,NaN,1160.0,21020.0,414.0,NaN,250.0,...,1111239,105,142,20618c231a4e05ea42a2a09221926ee50cd18565,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200819,AK,5247.0,305602.0,NaN,56.0,NaN,NaN,NaN,6.0,...,310849,0,0,85dcc331b8b35c88c030526eaedc78fce146bef2,0,0,0,0,0,NaN
1,20200819,AL,111478.0,773868.0,NaN,1198.0,13080.0,NaN,1336.0,NaN,...,885346,8,122,7095d1d867307ff5170d94a36cffbe87e3dbc113,0,0,0,0,0,NaN
2,20200819,AR,54216.0,587064.0,NaN,499.0,3743.0,NaN,NaN,114.0,...,641280,12,48,6a1605093a5772590825ab1faa2581c75ba85fef,0,0,0,0,0,NaN
3,20200819,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,c0f8fafd05d99956acbdcaa7c6515503d8e27f8f,0,0,0,0,0,NaN
4,20200819,AZ,195557.0,915682.0,NaN,1160.0,21020.0,414.0,NaN,250.0,...,1111239,105,142,20618c231a4e05ea42a2a09221926ee50cd18565,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-19,AK,5247,305602,NaN,56.0,NaN,NaN,NaN,6.0,NaN,...,310849,0,0,85dcc331b8b35c88c030526eaedc78fce146bef2,0,0,0,0,0,NaN
2020-08-19,AL,111478,773868,NaN,1198.0,13080.0,NaN,1336.0,NaN,731.0,...,885346,8,122,7095d1d867307ff5170d94a36cffbe87e3dbc113,0,0,0,0,0,NaN
2020-08-19,AR,54216,587064,NaN,499.0,3743.0,NaN,NaN,114.0,483.0,...,641280,12,48,6a1605093a5772590825ab1faa2581c75ba85fef,0,0,0,0,0,NaN
2020-08-19,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,c0f8fafd05d99956acbdcaa7c6515503d8e27f8f,0,0,0,0,0,NaN
2020-08-19,AZ,195557,915682,NaN,1160.0,21020.0,414.0,NaN,250.0,NaN,...,1111239,105,142,20618c231a4e05ea42a2a09221926ee50cd18565,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-19,AK,5247.0,305602.0,NaN,56.0,NaN,NaN,NaN,6.0,NaN,...,310849,0,0,85dcc331b8b35c88c030526eaedc78fce146bef2,0,0,0,0,0,NaN
2020-08-19,AL,111478.0,773868.0,NaN,1198.0,13080.0,NaN,1336.0,NaN,731.0,...,885346,8,122,7095d1d867307ff5170d94a36cffbe87e3dbc113,0,0,0,0,0,NaN
2020-08-19,AR,54216.0,587064.0,NaN,499.0,3743.0,NaN,NaN,114.0,483.0,...,641280,12,48,6a1605093a5772590825ab1faa2581c75ba85fef,0,0,0,0,0,NaN
2020-08-19,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,c0f8fafd05d99956acbdcaa7c6515503d8e27f8f,0,0,0,0,0,NaN
2020-08-19,AZ,195557.0,915682.0,NaN,1160.0,21020.0,414.0,NaN,250.0,NaN,...,1111239,105,142,20618c231a4e05ea42a2a09221926ee50cd18565,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE